# Terrenos vs otro
limpieza al shp de terrenos.
- Identificar los terrenos de casas y apendarlos con terrenos
- Agrupar las geometrias 
- Usar el archivo cruces_limpieza_shp


In [ ]:
# !pip install geopandas
import geopandas as gpd
import pandas as pd 
import sys
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
from codigos import obtener_curt

In [ ]:
#CORRER SOLO SI ETAS EN GOOGLE COLAB
def obtener_curt(data:gpd.GeoDataFrame, geom_col:str):
    '''
    (Function)
        Función que recibe un gdf y devuelve el mismo geodata con la clave unica de registro territorial
        en formato latitud(10 dígitos) + longitud(10 dígitos)
    (Parameters)
        - data    : GeoDataFrame al cual se le obtendrán las curt
        - geom_col: Str del nombre de la columna la cual contiene los polígonos 
    
    '''
    def convert_to_dms(coord):
        '''
    (Function)
        Funcion interna que recibe una coordenada y devuelve la misma pero en formato
        str de la forma: grados + minutos + segundos + diezmilesimas de segundo
    (Parameters)
        - coord: Coordenadas del centroide     
    '''
        degrees = int(coord)
        minutes = int((coord - degrees) * 60)
        seconds = int(((coord - degrees) * 60 - minutes) * 60)
        milliseconds = int((((coord - degrees) * 60 - minutes) * 60 - seconds) * 10000)
        return str(abs(degrees)).zfill(2)+str(abs(minutes)).zfill(2)+str(abs(seconds)).zfill(2)+str(abs(milliseconds)).zfill(4)

    data['centroid'] = data[geom_col].to_crs(4326).centroid
    data['lat'] = data['centroid'].y
    data['lon'] = data['centroid'].x
    data['CURT_f'] = data['lat'].apply(convert_to_dms)+data['lon'].apply(convert_to_dms)
    return data.drop(columns=['centroid','lat','lon'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Usar el de casas original
path_casas = "/content/drive/MyDrive/Geo/Data/geoshaps/Atlacomulco/Rotacion_Atlacomulco/prueba_deteccion_3_rotacion.shp"
# Usar terrenos original
path_terrenos = "/content/drive/MyDrive/Geo/Data/geoshaps/Atlacomulco/atlacomulco_3105/Salidas/Atlacomulco_terrenos.shp"
# Usar chinchetas original o el que ya se cruzó con denue
path_chinchetas = "/content/drive/MyDrive/Cruces_predial_otros/Atlacomulco/Bases_salida_limpieza/casas/Atlacomulco_casas_pinesxdenue.shp"
path_zonas = r"/content/drive/MyDrive/Geo/Data/bases_shp_INEGI/zonas_rurales_urbanas/15m.shp"
path_sia="/content/drive/MyDrive/Geo/Data/bases_shp_INEGI/bases_areasv_camellones/15sia.shp"
path_ejes = "/content/drive/MyDrive/Geo/Data/DELEGACIONES_EDOMEX/ATLACOMULCO_2018/ATLACOMULCO/Ejes_Atlacomulco.shp"

In [ ]:
#Carga de las bases
casas = gpd.read_file(path_casas)
terrenos = gpd.read_file(path_terrenos)
pines = gpd.read_file(path_chinchetas)
zonas = gpd.read_file(path_zonas)
sia = gpd.read_file(path_sia)
ejes = gpd.read_file(path_ejes)

In [ ]:
#Revisión de las etiquetas en la predicción de terrenos
terrenos.clase_dete.value_counts()

In [ ]:
#Revisión de las etiquetas en la predicción de casas
casas['clase_dete'].value_counts()

In [ ]:
#Crear una nueva base que contenga únicamente las detecciones de terrenos baldios en la predicción de casas
new_terrenos = casas[casas['clase_dete']== 'terreno_baldio']
new_terrenos.shape

In [ ]:
terrenos.columns

In [ ]:
#Asignarles la misma geometría para poder concatenarlos
terrenos=terrenos.set_crs(3857,allow_override=True)
new_terrenos=new_terrenos.set_crs(3857,allow_override=True)
print(terrenos.crs,new_terrenos.crs)

In [ ]:
# Concatenar terrenos de casas a terrenos
terrenos = pd.concat([terrenos.to_crs(3857), new_terrenos.to_crs(3857)])
terrenos.shape

In [ ]:
# Etiquetar las casas y establecimientos
#Agregar una columna para las detecciones en terrenos que contengan una etiqueta distinta a terreno_baldio para no perderlas
terrenos.loc[(terrenos['clase_dete']=='casas') | (terrenos['clase_dete']=='establecimiento'), 'con_const'] = 1
#Las que son terreno baldio se igualan a 0 para identificarlas
terrenos.loc[terrenos.con_const!=1, 'con_const'] = 0
#Se cambian las etiquetas q no son terreno baldio a terreno baldio
terrenos.loc[(terrenos['clase_dete']=='casas') | (terrenos['clase_dete']=='establecimiento'), 'clase_dete'] = 'terreno_baldio'

In [ ]:
#Asignar curt a la predicción de terrenos
terrenos=obtener_curt(terrenos,'geometry')
terrenos.tail(2)

In [ ]:
# Revision de duplicados por geometria
terrenos.geometry.value_counts()

In [ ]:
# Borrar duplicados
terrenos.sort_values('conf',ascending=False,inplace=True)
terrenos.drop_duplicates(subset='geometry',keep='first',inplace=True)
terrenos.tail(2)

In [ ]:
pines.columns

In [ ]:
#Eliminar las columnas que traen el concatenado de Clase y solo dejar la columna buena
pines.drop(['Clase1','Clase'],axis=1,inplace=True)
pines.tail(2)

In [ ]:
#La columna de clasificacion de chinchetas debe llamarse Clase, sino cambiar
pines.rename(columns={'Clase_lim':'Clase'},inplace=True)
pines.tail(2)

In [ ]:
#En el caso de ya tener pines vs denue, cambiar el nombre de la columna id a ID_denue
pines.rename(columns={'id':'ID_denue'},inplace=True)
pines.tail(2)

In [ ]:
pines.Clase.value_counts()

In [ ]:
pines.drop(['clase_dete'],axis=1,inplace=True)

In [ ]:
pines.rename(columns={'conf':'conf_pin','area':'area_pin'},inplace=True)

In [ ]:
# Solo quedarnos con los pines que pertenecen a Áreas verdes
pines_verdes = pines[(pines['Clase']== 'a_verdes')|(pines['Clase']== 'a_verdes_1') |(pines['Clase']== 'a_verdes_2')]
pines_verdes.shape

In [ ]:
#Cruce de terrenos x pines
cruce_terrenosxpines = gpd.sjoin(terrenos,pines_verdes)
cruce_terrenosxpines

In [ ]:
#Se agrega un identificador a los que si cruzan x pines
cruce_terrenosxpines['PIN_si'] = 1
cruce_terrenosxpines

In [ ]:
no_cruzanxPines=terrenos[~(terrenos.geometry.isin(cruce_terrenosxpines['geometry']))]
no_cruzanxPines.shape

In [ ]:
#Se agrega un identificador a los que no cruzan x pines
no_cruzanxPines['PIN_si']=0
no_cruzanxPines.tail(2)

In [ ]:
cruce_terrenosxpines.shape, no_cruzanxPines.shape

In [ ]:
#Se concatenan los que cruzaron con los que no
crucexpin=pd.concat([cruce_terrenosxpines,no_cruzanxPines])
crucexpin

In [ ]:
crucexpin.rename(columns={'index_right':'ID_chin'},inplace=True)

In [ ]:
crucexpin.columns

In [ ]:
#Visualización de base de zonas
zonas.tail(2)

In [ ]:
#Filtro por la clave del municipio en el que se está trabajando
zona=zonas[zonas.CVE_MUN=='014']
zona.plot()


In [ ]:
cruce_zona = gpd.sjoin(crucexpin,zona.to_crs(3857),how='left')
cruce_zona.rename(columns={'index_right':'ID_15m'},inplace=True)
cruce_zona.shape, crucexpin.shape

In [ ]:
cruce_zona.columns

In [ ]:
#Base de Areas verdes, camellones, glorieta
sia=sia.to_crs(3857)
sia.tail(2)

In [ ]:
#Cruzar con ejes para quitar calles con overlay 
# Ejes de calles
ejes = ejes.to_crs(3857)
ejes.tail(2)

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs ejes: ",ejes.crs)
print("crs sia: ",sia.crs)

In [ ]:
cruce_ejes = sia.sjoin(ejes)
print(cruce_ejes.shape)
'Porcentaje a borrar: ',cruce_ejes.shape[0]/sia.shape[0]*100

In [ ]:
import numpy as np 
print('Shape original: ', sia.shape)
index_drop = np.unique(sia[sia.geometry.isin(cruce_ejes.geometry)].index.values)
sia.drop(index=index_drop,inplace=True)
print('Shape final:    ', sia.shape)

In [ ]:
cruce_zona.shape, sia.shape

*Cruce de terrenos vs areas verdes (sia)*

In [ ]:
cruce_final = gpd.sjoin(cruce_zona,sia,how='left')
cruce_final.tail(2)

In [ ]:
#Combinar las columnas que se duplican porque están en ambas bases y crear una general
cruce_final['CVEGEO']     =cruce_final.CVEGEO_left.combine_first(    cruce_final.CVEGEO_right)
cruce_final['CVE_ENT']    =cruce_final.CVE_ENT_left.combine_first(   cruce_final.CVE_ENT_right)
cruce_final['CVE_MUN']    =cruce_final.CVE_MUN_left.combine_first(   cruce_final.CVE_MUN_right)
cruce_final['CVE_LOC']    =cruce_final.CVE_LOC_left.combine_first(   cruce_final.CVE_LOC_right)
cruce_final['AMBITO']     =cruce_final.AMBITO_left.combine_first(    cruce_final.AMBITO_right)
cruce_final.rename(columns={'index_right':'ID_sia'},inplace=True)
#Eliminar las columnas divididas porque ya están en la general
cruce_final.drop(['CVEGEO_left','CVEGEO_right'],axis=1,inplace=True)
cruce_final.drop(['CVE_ENT_left','CVE_ENT_right'],axis=1,inplace=True)
cruce_final.drop(['CVE_MUN_left','CVE_MUN_right'],axis=1,inplace=True)
cruce_final.drop(['CVE_LOC_left','CVE_LOC_right'],axis=1,inplace=True)
cruce_final.drop(['AMBITO_left','AMBITO_right'],axis=1,inplace=True)

In [ ]:
print('Columas de terrenos: ', cruce_final.columns)
if input('Quiere crear la columna ID_terreno') in ['1','si']:
    cruce_final.reset_index(drop=False, inplace=True)
    cruce_final.rename(columns={'index':'ID_terreno'},inplace=True)

In [ ]:
#revisar as==las columnas finales
cruce_final.columns

In [ ]:
#Acomodo de columnas finales
try:
    cruce_final = cruce_final[['ID_terreno', 'clase_dete', 'Clase','geometry','CURT_f', 'status',
                               'conf', 'area', 'con_const',  'ID_chin', 'conf_pin', 'area_pin', 'angulo',
                               'umbral', 'n_resize', 'tipoCenCom', 'tipo_asent', 'nombre_act',
                              'clee', 'ID_denue', 'PIN_si', 'ID_15m', 'CVE_AGEB', 'CVE_MZA',
                              'TIPOMZA', 'ID_sia', 'CONDICION', 'GEOGRAFICO', 'NOMSERV', 'TIPO',
                              'CVE_SERV', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'AMBITO']]
except:
    cruce_final = cruce_final[['ID_terreno', 'clase_dete', 'Clase','geometry','CURT_f', 'status',
                               'conf', 'area', 'con_const',  'ID_chin', 'conf_pin', 'area_pin', 'angulo',
                               'umbral', 'n_resize', 'tipoCenCom', 'tipo_asent', 'nombre_act',
                              'clee', 'ID_denue', 'PIN_si', 'ID_15m', 'CVE_AGEB', 'CVE_MZA',
                              'TIPOMZA', 'ID_sia', 'CONDICION', 'GEOGRAFICO', 'NOMSERV', 'TIPO',
                              'CVE_SERV', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'AMBITO']]

In [ ]:
cruce_final[cruce_final.ID_terreno.isna()]

In [ ]:
#Guardar base final
cruce_final.to_file('/content/drive/MyDrive/Cruces_predial_otros/Atlacomulco/Bases_salida_limpieza/terrenos/Atlacomulco_terrenos_cruces_todos.shp',
                 index=False)